<a href="https://colab.research.google.com/github/PhrygianDeva/Chatbot_ML/blob/master/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import numpy as np
import json
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Embedding , GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.preprocessing import LabelEncoder

In [22]:
with open('/content/intents.json') as file:
  data = json.load(file)

training_sentences=[]
training_labels=[]
labels=[]
responses=[]

for intent in data['intents']:
  for pattern in intent['patterns']:
    training_sentences.append(pattern)
    training_labels.append(intent['tag'])
  responses.append(intent['responses'])

  if intent['tag'] not in labels:
    labels.append(intent['tag'])

  num_classes=len(labels)

num_classes

8

In [23]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)

training_labels=lbl_encoder.transform(training_labels)


In [24]:
vocab_size = 1000
embedding_dim=16
max_len=20
oov_token="<00V>"

tokenizer=Tokenizer(num_words=vocab_size,oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index=tokenizer.word_index

sequences=tokenizer.texts_to_sequences(training_sentences)
padded_sequences=pad_sequences(sequences,truncating="post",maxlen=max_len)


In [26]:
model=Sequential()

model.add(Embedding(vocab_size,embedding_dim,input_length=max_len))

model.add(GlobalAveragePooling1D())

model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes,activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam' , metrics=['accuracy'])

model.summary()
epochs=20
history = model.fit(padded_sequences,np.array(training_labels),epochs=epochs)


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d_4  (None, 16)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_8 (Dense)             (None, 16)                272       
                                                                 
 dense_9 (Dense)             (None, 8)                 136       
                                                                 
Total params: 16408 (64.09 KB)
Trainable params: 16408 (64.09 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/20
2/2 [==============================] - 1s 8ms/step - loss: 2.0782 - accur

In [27]:
model.save("chat")

import pickle

with open('tokenizer.pickle' , 'wb') as handle:
  pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)

with open('label_encoder.pickle','wb') as ecn_file:
  pickle.dump(lbl_encoder,ecn_file,protocol=pickle.HIGHEST_PROTOCOL)


In [28]:
!pip install colorama
import colorama

colorama.init()

from colorama import Fore, Style , Back

import random

In [29]:
with open('/content/intents.json') as file:
  data=json.load(file)

def chat():
  model=keras.models.load_model('chat')

  with open('tokenizer.pickle','rb') as handle:
    tokenizer=pickle.load(handle)

  with open('label_encoder.pickle','rb') as enc:
    lbl_encoder=pickle.load(enc)

    max_len=20

  while True:
    print(Fore.LIGHTBLUE_EX + "User:" + Style.RESET_ALL,end="")
    inp=input()
    if inp.lower()=="quit":
      break

    result=model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                                                    truncating='post',maxlen=max_len))

    tag = lbl_encoder.inverse_transform([np.argmax(result)])

    for i in data['intents']:
      if i['tag']==tag:
        print(Fore.GREEN+ "ChatBot:" + Style.RESET_ALL,np.random.choice(i['responses']))

print(Fore.YELLOW+ "Start message" + Style.RESET_ALL)

chat()


Start message
User:Hi
1/1 [==============================] - 0s 120ms/step
ChatBot: Hi
User:yay
1/1 [==============================] - 0s 25ms/step
ChatBot: Hi
User:ke
1/1 [==============================] - 0s 27ms/step
ChatBot: Hi there
User:thank you
1/1 [==============================] - 0s 28ms/step
ChatBot: Hi
User:Hey
1/1 [==============================] - 0s 26ms/step
ChatBot: Hi there
User:Help me
1/1 [==============================] - 0s 27ms/step
ChatBot: Hi
User:Could you help me?
1/1 [==============================] - 0s 25ms/step
ChatBot: Hi
User:quit
